In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.utils.np_utils import to_categorical
from keras.layers import Convolution2D, MaxPooling2D, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

## Učenje modela s CNN Model ###

#### Najprej naložimo podatke in jih razdelimo na testno in učno množico ter spremenimo format podatkov.


In [2]:
train = pd.read_csv('podatki/train.csv')
test = pd.read_csv('podatki/test.csv')
X_train = (train.iloc[:, 1:].values).astype('float32')  #vsi pixli
Y_train = train.iloc[:, 0].values.astype('int32') #target stevilke
X_test = test.values.astype('float32')


                        #(slika, #st vrstic, #st stolpcev, #barva)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)


#### Podatke standardiziramo, tako podatke centriramo okoli varijance in povprečja, z funkcijo "stand"

In [3]:
mean_pixl = X_train.mean().astype(np.float32)
std_pikl = X_train.std().astype(np.float32)
def stand(x):
    return (x-mean_pixl)/std_pikl


#### Uporabimo "One-Hot encoding" da določimo števila od 0 do 9. Primer: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]  predstavlja številko 5. Torej tam kjer je bit 1 vzamemo indeks in dobimo številko. 

In [4]:
import tensorflow as tf
from keras.utils.np_utils import to_categorical
Y_train = to_categorical(Y_train)
num_of_classes = Y_train.shape[1]
num_of_classes

#### CNN - Convolution Neural Network, so najbolj uporabne pri procesiranju slik, zato smo jih tudi uporabli za naš model.


In [5]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

def cnn_model():
    model = Sequential([
        Lambda(stand, input_shape=(28, 28, 1)),
        Convolution2D(32, (3, 3), activation='relu'),
        Convolution2D(32, (3, 3), activation='relu'),
        MaxPooling2D(),
        Convolution2D(64, (3, 3), activation='relu'),
        Convolution2D(64, (3, 3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = cnn_model()

## Grafični vmesnik za prepoznavanje števil

Poleg testnih podatkov iz Kaggla smo dodali grafični vmesnik za prepoznavanje števil, ki temelji na knjižnici PyQt5. Vanj lahko z uporabo miške napišemo želeno število, program pa ga na podlagi vnaprej naučenega modela poskuša prepoznati in izpiše natančnost te ocene.

[Slika vmesnika](screen_GUI.png)

Za pridobivanje slike iz grafičnega vmesnika je narisano sliko potrebno shraniti, jo pomanjšati na velikost 28×28 in zmanjšati globino slike iz 24 na 8 bitov (256&nbsp;barv). Tak zapis je primeren za napoved z vnaprej naučenim modelom.

In [6]:
def get_image():
    img = window.label.grab().toImage().save('../podatki/num.jpg')
    img = Image.open('../podatki/num.jpg')
    img = img.resize((28, 28))
    img = img.convert('L')
    img_array = np.array(img)
    pred, acc = predict_GUI_number(model, img_array)